---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [30]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

def blight_model():
    
    # Read in data sets
    train = pd.read_csv('train.csv', engine = 'python')
    test = pd.read_csv('test.csv')
    address = pd.read_csv('addresses.csv')
    geo = pd.read_csv('latlons.csv')
    
    # Merge/join the address and geo data. Then merge the resultant with train/test data.
    geo_address = pd.merge(address, geo, on='address')
    train2 = pd.merge(geo_address, train, on='ticket_id')
    test2 = pd.merge(geo_address, test, on='ticket_id')
    
    # Remove null values in target variable in training set, since they are not available
    # in test set.
    train3 = train2.dropna(subset=['compliance'])
    
    # Drop features not available in the test set.
    train4 = train3.drop(['payment_amount', 'payment_date', 'balance_due', 
                          'collection_status', 'compliance_detail', 
                          'payment_status'], axis=1)
    
    # Drop unnecessary categorical/string data.
    def column_drop(data_set):
        """
        This function simply drops unnecessary columns from a dataset.
        """
        column_list = ['address', 'agency_name', 'inspector_name', 'violator_name',
                       'violation_street_number', 'violation_street_name',
                       'violation_zip_code', 'mailing_address_str_number',
                       'mailing_address_str_name', 'city', 'state',
                       'non_us_str_code', 'country', 'ticket_issued_date', 
                       'hearing_date', 'violation_code','violation_description', 
                       'clean_up_cost', 'grafitti_status', 'zip_code']
        return data_set.drop(column_list, axis=1)
    
    train5 = column_drop(train4)
    test3 = column_drop(test2)
    
    """
    One engineered feature that may have been interesting to include is time of day
    or day of the week, but continue without them for now.
    """
    
    # Label encode disposition feature assuming the feature is ordinal
    encode = LabelEncoder()
    encode.fit(train5['disposition'].append(test['disposition']).reshape(-1,1))
    train5['disposition'] = encode.transform(train5['disposition'])
    test3['disposition'] = encode.transform(test3['disposition'])
    
    train6 = train5.set_index('ticket_id')
    test3 = test3.set_index('ticket_id')
    
    # Use fillna() to fill lat/lon columns, since there are null values.
    mean_lat = train6['lat'].mean()
    mean_lon = train6['lon'].mean()
    
    train6['lat'].fillna(mean_lat, inplace=True)
    train6['lon'].fillna(mean_lon, inplace=True)
    test3['lat'].fillna(mean_lat, inplace=True)
    test3['lon'].fillna(mean_lon, inplace=True) 
    
    # Create X and y for training.
    X_train = train6.drop('compliance', axis=1)
    y_train = train6['compliance'].copy()
    
        
    #Scale data so that all algorithms function optimally.
    #Do not refit the scaler to test data. Transform test data only.
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(test3)
    
    # Data is processed. Start with Logistic Regression classifier
#     log_clf = LogisticRegression()
    
#     #Use gridsearch to find optimal parameter value.
#     log_grid_values = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
    
#     log_grid_search = GridSearchCV(log_clf, log_grid_values, cv=3, scoring='roc_auc')
#     log_grid_search.fit(X_train_scaled, y_train) 
#     print(log_grid_search.best_score_)
    
    # Try random forest classifier    
    rf_clf = RandomForestClassifier()
    
    grid_values = {'n_estimators': [5, 20], 'max_depth': [5, 10, 20]}
    
    rf_grid_search = GridSearchCV(rf_clf, grid_values, scoring='roc_auc')
    rf_grid_search.fit(X_train_scaled, y_train.values.reshape(-1,))
    #print(rf_grid_search.best_score_)
    
    final = pd.DataFrame(rf_grid_search.predict_proba(X_test_scaled)[:,1], 
                         test['ticket_id'], columns='compliance proba')
    
    return final

blight_model()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:63: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,0
ticket_id,
284932,0.136497
285362,0.135933
285361,0.050409
285338,0.101819
285346,0.032078
285345,0.028827
285347,0.028827
285342,0.063045
285530,0.187778
